<a href="https://colab.research.google.com/github/gibsonx/AnsibleGUI/blob/master/Federated_Learning_DockerPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Modified by Gibson Xue on July 25, 2021
!pip uninstall --yes tensorflow tensorboard tb-nightly
!pip install --quiet tensorflow==2.5.0
!pip install --quiet tensorflow-federated==0.19.0
!pip install --quiet nest-asyncio
!pip install --quiet tensorboard  # or tensorboard, but not both
!pip install keras
import tensorflow as tf

import nest_asyncio
nest_asyncio.apply()
print(tf.__version__)

from tensorflow_federated.python.simulation import FileCheckpointManager
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd

Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
Uninstalling tensorboard-2.5.0:
  Successfully uninstalled tensorboard-2.5.0
     |████████████████████████████████| 454.3MB 37kB/s 
     |████████████████████████████████| 6.0MB 41.9MB/s 
     |████████████████████████████████| 604kB 32.6MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 194kB 51.4MB/s 
     |████████████████████████████████| 174kB 48.6MB/s 
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.28.1 which is incompatible.
ERROR: pymc3 3.11.2 has requirement cachetools>=4.2.1, but you'll have cachetools 3.1.1 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement tqdm>=4.36.1, but you'll have tqdm 4.28.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
2.5.0


In [8]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/train.csv /content/train.csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# example_dataset = emnist_train.create_tf_dataset_for_client(
#     emnist_train.client_ids[0])
# for i in example_dataset.take(1):
#   print(i)
data = pd.read_csv('train.csv')
train_data = data.head(37800)
val_data = data.tail(4200)
train_labels = train_data.pop('label')
val_labels = val_data.pop('label')

tf_train_data = tf.data.Dataset.from_tensor_slices((train_data.values, train_labels.values))
tf_val_data = tf.data.Dataset.from_tensor_slices((val_data.values, val_labels.values))
print(tf_train_data)
print(tf_val_data)
# print(type(example_dataset))

# import tensorflow_datasets as tfds

# np.savetxt("foo.csv", a, delimiter=",")
# import numpy as geek

# list_train = []
# len(emnist_train.client_ids)
# for i in range(0,1):
#   client_dataset = emnist_train.create_tf_dataset_for_client(
#       emnist_train.client_ids[i])
#   print(client_dataset.element_spec)
#   for example in client_dataset:
#     pixels = example['pixels'].numpy()
#     label = geek.array_str(example['pixels'].numpy())
#     list_train.append({'pixels': pixels, 'label': label, 'client_id': i, 'is_checked': 0 })
    # # t = geek.array(g)
    # print(pixels)
    # break

# list_train
# df = pd.DataFrame(list_train)
# df.to_csv('data.csv',index=False)
# compression_opts = dict(method='zip',archive_name='data.csv')
# df.to_csv(data.csv, index=False,
#           compression=compression_opts)


<TensorSliceDataset shapes: ((784,), ()), types: (tf.int64, tf.int64)>
<TensorSliceDataset shapes: ((784,), ()), types: (tf.int64, tf.int64)>


In [ ]:
# df = pd.read_csv('data.csv')
# dataset = tf.data.experimental.CsvDataset('data.csv',[tf.int32],select_cols=[1])

# d = tf.convert_to_tensor(g,dtype=tf.float32)
# d
# for i in dataset:
#   print(i)
# number_column = df.loc[:,'pixels'].apply(lambda x : tf.convert_to_tensor(x, dtype=tf.float32))
# number_column
# df = tf.data.experimental.make_csv_dataset('data.csv',batch_size=2)
# for i in number_column:
#   print(i)
#   break
# example_dataset = emnist_train.create_tf_dataset_for_client(
#     emnist_train.client_ids[0])

# example_dataset

# preprocessed_example_dataset = preprocess(df.to_numpy())

# preprocessed_example_dataset = preprocess(example_dataset)

In [10]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 784]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [11]:
def preprocess_image(image, label):
    image = tf.reshape(image, [28, 28, 1])
    image = tf.cast(image, tf.float32) / 255.
    
    return image, label

tf_train_data = tf_train_data.map(
    preprocess_image, 
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)

tf_val_data = tf_val_data.map(
    preprocess_image, 
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)

print(tf_train_data)
print(tf_val_data)

# tf_train_data = pipeline(tf_train_data)
# tf_val_data = pipeline(tf_val_data)


<ParallelMapDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>
<ParallelMapDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>


In [12]:
def pipeline(tf_data):
    tf_data = tf_data.shuffle(100)
    tf_data = tf_data.batch(32)
    tf_data = tf_data.prefetch(tf.data.experimental.AUTOTUNE)
    
    return tf_data

tf_train_data = pipeline(tf_train_data)
tf_val_data = pipeline(tf_val_data)
print(tf_train_data)
print(tf_val_data)

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>
<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>


In [13]:
def create_keras_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, (5, 5), activation='relu', padding='same', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(16, (5, 5), activation='relu', padding='valid'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(84, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
  ])

def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=tf_train_data.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [14]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [15]:
# state = iterative_process.initialize()
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, verbose=1),
]

optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)

model = create_keras_model()

model.compile(
    optimizer=optimiser, 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])


train_log = model.fit(
    tf_train_data,
    validation_data=tf_val_data,
    epochs=30,
    callbacks=callbacks
)

Epoch 1/30
 180/1182 [===>..........................] - ETA: 18s - loss: 0.7936 - accuracy: 0.7599

KeyboardInterrupt: ignored

In [30]:
state = iterative_process.initialize()
# state, metrics = iterative_process.next(state, tf_train_data)
# state
# for i in tf_train_data:
#   print(collections.OrderedDict(x=i))
#   break
tf_train_data.element_spec


(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))